## NOTES:
<ul>
<li>Sections labeled <strong>CORE</strong> must be run in order to begin Aquifer Analysis</li>
<li>Cells labeled <strong>Control</strong> contain inputs for the immeadiately proceeding section(s)</li>
<li>Sections labeled <strong>EXTRA</strong> contain additional plotting or analysis tools but are not necessary for Aquifer Analysis</li>
</ul>

## CORE: Imports

In [22]:
#Python3.10
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import contextily as cx
import requests
import calendar
from importlib import reload
from typing import IO

from datetime import datetime, timedelta
from shapely.geometry import Point
from io import StringIO
from mpl_toolkits.axes_grid1 import make_axes_locatable

# USGS Data retreival tool
from dataretrieval import nwis, utils, codes

# Custom modules are imported in multiple locations to faciliate easy reloading when edits are made to their respective files
import Src.classes as cl
import Src.func as fn
reload(cl)
reload(fn)

# TODO: Look into the warning that this is disabling. It doesn't appear to be significant for the purposes of this code but should be understood
pd.options.mode.chained_assignment = None

#pd.options.mode.chained_assignment = 'warn'

## CORE: Single Site Data<br>
This function produces the streamflow analysis (using the functions above) for a single site given a list of thresholds and time windows to analyze

Controls:

In [2]:
import Src.classes as cl
import Src.func as fn
reload(cl)

# df2 holds all-time data, df is analyzed range
curr_guage = cl.SRB_Guage
df = nwis.get_record(sites='11447650', service=fn.SERVICE, parameterCD=[fn.PARAM_CODE, fn.TIDAL_CODE], start=fn.DEFAULT_START, end=fn.DEFAULT_END)
df = fn.merge_tidal(df)
#df2 = nwis.get_record(sites=curr_guage.id, service=fn.SERVICE, parameterCD=fn.PARAM_CODE, start=fn.DEFAULT_START, end='2014-09-30')


# Set to true if running indepedently to verify data with Kocis paper (or other source)
# Set to false if running Aquifer Analysis
testing = False

Single Site Analysis (MEAN):

In [9]:
# If looking at a post impairment period, but calculating threshold based on the full record of data, pass a second dataframe with a different 
# start/end date as the final parameter. This method was used in Kocis 2017 and is needed for some data verification, but is not the methodology
# used for the Aquifer Analysis and so *args will most often be empty.
import Src.func as fn
reload(fn)

def single_site_data(df: pd.DataFrame, quantiles_list: list, data_ranges_list: list, *args):
    df = df.reset_index()    
    threshold = None
    df_complete_site_data = pd.DataFrame()    
    df_complete_mk_mag = pd.DataFrame()
    df_complete_mk_dur = pd.DataFrame()
    df_complete_mk_intra = pd.DataFrame()
    df_complete_mk_event_mag = pd.DataFrame()
    df_complete_mk_event_dur = pd.DataFrame()
    df_complete_mk_timing = pd.DataFrame()
    
    for data_range in data_ranges_list:
        for quantile in quantiles_list:
            # Copying original dataframe to avoid any conflicts
            df_copy = df.copy()
            
            # Validate that site is not missing > 10% of data
            date_threshold = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * data_range)              
            
            # Filter dataframe based on current data_range (don't do this during testing if testing unique dataset/range)
            if not testing:
                df_copy = df_copy[df_copy['datetime'].dt.date >= date_threshold]
            
            # Validate <10% missing data based on filtered dataframe
            missing = fn.validate(df_copy, date_threshold, fn.DEFAULT_END)
            valid = missing < fn.MAX_MISSING_THRESHOLD
            missing = round(missing, 5) * 100                              
    
            # Check for optional second dataframe containing full record
            if args and isinstance(args[0], pd.DataFrame) and not args[0].empty:
                #print('Threshold calculation across the full record')
                df2 = args[0].reset_index()
                threshold = fn.calc_threshold(df2, quantile)
                # threshold = 52350 # SRB_Guage post-impairment threshold for verification with Kocis_2017 
            else:
                #print('Threshold calculation across a limited record')
                threshold = fn.calc_threshold(df_copy, quantile)                             

            # Create a dataframe with only days over HMF threshold as well as continuous dataframe for MK trend test
            hmf_series_defl, hmf_series_cont = fn.filter_hmf(df_copy, threshold)
            #print(zero_deflated_hmf)
            
            # Aggregate data before performing MK magnitude test
            df_agg_cont = hmf_series_cont.copy()
            df_agg_cont = fn.convert_hmf(df_agg_cont, threshold)
            df_agg_cont['00060_Mean'] = df_agg_cont['00060_Mean'].apply(lambda x: x * fn.CUBIC_FT_KM_FACTOR if x >= 0 else 0)
            df_agg_cont.set_index('datetime', inplace=True)
            df_agg_cont = df_agg_cont.resample(fn.HYDRO_YEAR).agg({'00060_Mean': ['sum', 'count']})
            df_agg_cont.columns = ['Sum', 'Count']
            df_agg_cont_defl = df_agg_cont[df_agg_cont['Sum'] > 0]          
            
            # MK Magnitude
            df_mk_mag = fn.mann_kendall(df_agg_cont_defl['Sum'], df_agg_cont['Sum'], fn.MK_TREND_ALPHA)

            # Find number of years with HMF
            hmf_years = fn.num_hmf_years(hmf_series_defl)            

            # Mask out months that don't fall within 3 and 6 month Winter range
            df_six_month, df_three_month = fn.three_six_range(hmf_series_defl, 12, 2, 11, 4)

            # Convert to daily average flow in cfpd, and take only flow above the threshold
            hmf_series_defl = fn.convert_hmf(hmf_series_defl, threshold)
            total_hmf_flow = hmf_series_defl["00060_Mean"].sum()
            hmf_per_month = fn.monthly_hmf(hmf_series_defl, data_range, quantile)

            # Calculate 3 and 6 month HMF
            df_six_month = fn.convert_hmf(df_six_month, threshold)
            six_month_hmf = df_six_month["00060_Mean"].sum()
            df_three_month = fn.convert_hmf(df_three_month, threshold)
            three_month_hmf = df_three_month["00060_Mean"].sum()
            
            total_hmf_flow = (total_hmf_flow * fn.CUBIC_FT_KM_FACTOR) / hmf_years
            six_month_hmf = (six_month_hmf * fn.CUBIC_FT_KM_FACTOR) / hmf_years
            three_month_hmf = (three_month_hmf * fn.CUBIC_FT_KM_FACTOR) / hmf_years

            # Inter-annual
            inter_annual, delta = fn.calc_inter_annual(hmf_series_cont, hmf_years)            

            # Average Duration and Intra-annual Frequency
            hmf_series_cont = fn.convert_hmf(hmf_series_cont, threshold)
            event_duration, annual_duration, intra_annual, event_hmf, df_results = fn.calc_duration_intra_annual(hmf_series_cont, hmf_years)
            
            # Total duration mk test 
            dur_series_defl = df_results['total_days'][df_results['total_days'] > 0]
            dur_series_cont = df_results['total_days']
            df_mk_dur = fn.mann_kendall(dur_series_defl, dur_series_cont, fn.MK_TREND_ALPHA)
                        
            # Total events per year mk test            
            intra_series_defl = df_results['total_events'][df_results['total_events'] > 0]
            intra_series_cont = df_results['total_events']
            df_mk_intra = fn.mann_kendall(intra_series_defl, intra_series_cont, fn.MK_TREND_ALPHA) 
            
            # Average event HMF mk test
            event_mag_series_defl = df_results['event_hmf'][df_results['event_hmf'] > 0]
            event_mag_series_cont = df_results['event_hmf']
            df_mk_event_mag = fn.mann_kendall(event_mag_series_defl, event_mag_series_cont, fn.MK_TREND_ALPHA)
            
            # Average event duration mk test
            event_dur_series_defl = df_results['duration'][df_results['duration'] > 0]
            event_dur_series_cont = df_results['duration']
            df_mk_event_dur = fn.mann_kendall(event_dur_series_defl, event_dur_series_cont, fn.MK_TREND_ALPHA)                
            
            # Timing Calculation using DOHY
            timing, com_series = fn.calc_timing(hmf_series_defl)
            df_mk_timing = fn.mann_kendall(com_series, com_series, fn.MK_TREND_ALPHA)   
                     
            
            # Merging site dataframe with Mann-Kendall dataframe. This start date is the beginning of the actual data, not necessarily 
            # the beginning of the analyzed range. Validation (above) starts from the start of the official range (1970/90-2020)
            start = df_copy['datetime'].min().date()
            end = df_copy['datetime'].max().date()
            data = {'dataset_ID': (data_range * quantile), 'site_no': df_copy.iloc[0]['site_no'], 'analyze_start': start, 'analyze_end': end, 'analyze_range': delta, 'quantile': quantile, 
                    'valid': valid, 'missing_data%': missing, 'threshold': threshold, 'hmf_years': hmf_years, 'annual_hmf': total_hmf_flow, 'six_mo_hmf': six_month_hmf, 'three_mo_hmf': three_month_hmf, 
                    'annual_duration': annual_duration, 'event_duration': event_duration, 'event_hmf': event_hmf, 'inter_annual%': inter_annual, 'intra_annual': intra_annual, 'timing': timing}              
            
            # Merging MK results
            df_complete_mk_mag = fn.merge_mk_results(df_complete_mk_mag, df_mk_mag, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_dur = fn.merge_mk_results(df_complete_mk_dur, df_mk_dur, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_intra = fn.merge_mk_results(df_complete_mk_intra, df_mk_intra, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_event_mag = fn.merge_mk_results(df_complete_mk_event_mag, df_mk_event_mag, df_copy.iloc[0]['site_no'], data_range, quantile) 
            df_complete_mk_event_dur = fn.merge_mk_results(df_complete_mk_event_dur, df_mk_event_dur, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_timing = fn.merge_mk_results(df_complete_mk_timing, df_mk_timing, df_copy.iloc[0]['site_no'], data_range, quantile)        
            
            # Merging metric results
            df_single_iteration = pd.DataFrame(data, index=['0'])
            df_single_iteration = pd.concat([df_single_iteration.reset_index(drop=True), hmf_per_month.reset_index(drop=True)], axis=1)
            df_complete_site_data = pd.concat([df_complete_site_data.reset_index(drop=True), df_single_iteration.reset_index(drop=True)], axis=0)
        
    return df_complete_site_data, df_complete_mk_mag, df_complete_mk_dur, df_complete_mk_intra, df_complete_mk_event_mag, df_complete_mk_event_dur, df_complete_mk_timing

# For testing purposes, to run this cell independently
single_site_result = single_site_data(df, fn.QUANTILE_LIST, fn.DATA_RANGE_LIST)

'''
try:
    with pd.ExcelWriter('df_single_site.xlsx') as writer:
        df_complete_site_data.to_excel(writer, sheet_name='site_metrics', index=False)
        df_complete_mk_mag.to_excel(writer, sheet_name='mk_magnitude', index=False)
        df_complete_mk_intra.to_excel(writer, sheet_name='mk_intra', index=False)
        df_complete_mk_dur.to_excel(writer, sheet_name='mk_duration', index=False)
except Exception as e:
    print(e)
'''
   
fn.single_site_report(single_site_result[0])
df_complete_site_data = fn.gages_2_filtering(single_site_result[0])
#fn.save_data(df_complete_site_data, df_complete_mk_mag, df_complete_mk_dur, df_complete_mk_intra, 'TEST')


Site No: 0    11447650
1    11447650
2    11447650
0    11447650
Name: site_no, dtype: object
Analyzed Range: 30
30
50
50
Quantile: 0.90
0.95
0.90
0.95
Valid: True
True
True
True
% Missing: 0
0
0
0
90%: 48800.0
69000.0
50600.0
69500.0
HMF Years: 20
15
33
27
Annual Duration: 54.600000
36.133333
55.090909
33.814815
Event Duration: 20.760833
14.422222
21.529293
12.409171
Event HMF: 1.033732
0.290248
1.045558
0.260939
Inter-annual Frequency: 66.667
50.000
66.000
54.000%
Intra-annual Frequency: 2.700000
2.600000
2.666667
2.777778
Total HMF in km^3/year: 2.632442
0.751311
2.585712
0.719230
Center of Mass: 144.600000
159.000000
140.636364
147.370370
6 Month HMF in km^3/year: 2.440887
0.717613
2.449815
0.699585
3 Month HMF in km^3/year: 1.373055
0.484194
1.399463
0.465339


Single Site Analysis (MEDIAN):

In [3]:
import Src.func as fn
reload(fn)

def single_site_data(df: pd.DataFrame, quantiles_list: list, data_ranges_list: list, *args):
    df = df.reset_index()    
    threshold = None
    df_complete_site_data = pd.DataFrame()    
    df_complete_mk_mag = pd.DataFrame()
    df_complete_mk_dur = pd.DataFrame()
    df_complete_mk_intra = pd.DataFrame()
    df_complete_mk_event_mag = pd.DataFrame()
    df_complete_mk_event_dur = pd.DataFrame()
    df_complete_mk_timing = pd.DataFrame()
    
    for data_range in data_ranges_list:
        for quantile in quantiles_list:
            # Copying original dataframe to avoid any conflicts
            df_copy = df.copy()
            
            # Validate that site is not missing > 10% of data
            date_threshold = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * data_range)              
            
            # Filter dataframe based on current data_range (don't do this during testing if testing unique dataset/range)
            if not testing:
                df_copy = df_copy[df_copy['datetime'].dt.date >= date_threshold]
            
            # Validate <10% missing data based on filtered dataframe
            missing = fn.validate(df_copy, date_threshold, fn.DEFAULT_END)
            valid = missing < fn.MAX_MISSING_THRESHOLD
            missing = round(missing, 5) * 100                              
    
            # Check for optional second dataframe containing full record
            if args and isinstance(args[0], pd.DataFrame) and not args[0].empty:
                #print('Threshold calculation across the full record')
                df2 = args[0].reset_index()
                threshold = fn.calc_threshold(df2, quantile)
                # threshold = 52350 # SRB_Guage post-impairment threshold for verification with Kocis_2017 
            else:
                #print('Threshold calculation across a limited record')
                threshold = fn.calc_threshold(df_copy, quantile)                             

            # Create a dataframe with only days over HMF threshold as well as continuous dataframe for MK trend test
            hmf_series_defl, hmf_series_cont = fn.filter_hmf(df_copy, threshold)
            #print(zero_deflated_hmf)
            
            # Aggregate data before performing MK magnitude test
            df_agg_cont = hmf_series_cont.copy()
            df_agg_cont = fn.convert_hmf(df_agg_cont, threshold)
            df_agg_cont['00060_Mean'] = df_agg_cont['00060_Mean'].apply(lambda x: x * fn.CUBIC_FT_KM_FACTOR if x >= 0 else 0)
            df_agg_cont.set_index('datetime', inplace=True)
            df_agg_cont = df_agg_cont.resample(fn.HYDRO_YEAR).agg({'00060_Mean': ['sum', 'count']})
            df_agg_cont.columns = ['Sum', 'Count']
            df_agg_cont_defl = df_agg_cont[df_agg_cont['Sum'] > 0]          
            
            # MK Magnitude
            df_mk_mag = fn.mann_kendall(df_agg_cont_defl['Sum'], df_agg_cont['Sum'], fn.MK_TREND_ALPHA)

            # Find number of years with HMF
            hmf_years = fn.num_hmf_years(hmf_series_defl)            

            # Mask out months that don't fall within 3 and 6 month Winter range
            df_six_month, df_three_month = fn.three_six_range(hmf_series_defl, 12, 2, 11, 4)

            # Convert to daily average flow in cfpd, and take only flow above the threshold
            hmf_series_defl = fn.convert_hmf(hmf_series_defl, threshold)
            total_hmf_flow = hmf_series_defl["00060_Mean"].median() * fn.CUBIC_FT_KM_FACTOR
            hmf_per_month = fn.monthly_hmf(hmf_series_defl, data_range, quantile)

            # Calculate 3 and 6 month HMF
            df_six_month = fn.convert_hmf(df_six_month, threshold)
            six_month_hmf = df_six_month["00060_Mean"].median() * fn.CUBIC_FT_KM_FACTOR
            df_three_month = fn.convert_hmf(df_three_month, threshold)
            three_month_hmf = df_three_month["00060_Mean"].median() * fn.CUBIC_FT_KM_FACTOR
            
            # total_hmf_flow = (total_hmf_flow * fn.CUBIC_FT_KM_FACTOR) / hmf_years
            # six_month_hmf = (six_month_hmf * fn.CUBIC_FT_KM_FACTOR) / hmf_years
            # three_month_hmf = (three_month_hmf * fn.CUBIC_FT_KM_FACTOR) / hmf_years

            # Inter-annual
            inter_annual, delta = fn.calc_inter_annual(hmf_series_cont, hmf_years)            

            # Average Duration and Intra-annual Frequency
            hmf_series_cont = fn.convert_hmf(hmf_series_cont, threshold)
            event_duration, annual_duration, intra_annual, event_hmf, df_results = fn.calc_duration_intra_annual(hmf_series_cont, hmf_years)
            
            # Total duration mk test 
            dur_series_defl = df_results['total_days'][df_results['total_days'] > 0]
            dur_series_cont = df_results['total_days']
            df_mk_dur = fn.mann_kendall(dur_series_defl, dur_series_cont, fn.MK_TREND_ALPHA)
                        
            # Total events per year mk test            
            intra_series_defl = df_results['total_events'][df_results['total_events'] > 0]
            intra_series_cont = df_results['total_events']
            df_mk_intra = fn.mann_kendall(intra_series_defl, intra_series_cont, fn.MK_TREND_ALPHA) 
            
            # Average event HMF mk test
            event_mag_series_defl = df_results['event_hmf'][df_results['event_hmf'] > 0]
            event_mag_series_cont = df_results['event_hmf']
            df_mk_event_mag = fn.mann_kendall(event_mag_series_defl, event_mag_series_cont, fn.MK_TREND_ALPHA)
            
            # Average event duration mk test
            event_dur_series_defl = df_results['duration'][df_results['duration'] > 0]
            event_dur_series_cont = df_results['duration']
            df_mk_event_dur = fn.mann_kendall(event_dur_series_defl, event_dur_series_cont, fn.MK_TREND_ALPHA)                
            
            # Timing Calculation using DOHY
            timing, com_series = fn.calc_timing(hmf_series_defl)
            df_mk_timing = fn.mann_kendall(com_series, com_series, fn.MK_TREND_ALPHA)   
                     
            
            # Merging site dataframe with Mann-Kendall dataframe. This start date is the beginning of the actual data, not necessarily 
            # the beginning of the analyzed range. Validation (above) starts from the start of the official range (1970/90-2020)
            start = df_copy['datetime'].min().date()
            end = df_copy['datetime'].max().date()
            data = {'dataset_ID': (data_range * quantile), 'site_no': df_copy.iloc[0]['site_no'], 'analyze_start': start, 'analyze_end': end, 'analyze_range': delta, 'quantile': quantile, 
                    'valid': valid, 'missing_data%': missing, 'threshold': threshold, 'hmf_years': hmf_years, 'annual_hmf': total_hmf_flow, 'six_mo_hmf': six_month_hmf, 'three_mo_hmf': three_month_hmf, 
                    'annual_duration': annual_duration, 'event_duration': event_duration, 'event_hmf': event_hmf, 'inter_annual%': inter_annual, 'intra_annual': intra_annual, 'timing': timing}              
            
            # Merging MK results
            df_complete_mk_mag = fn.merge_mk_results(df_complete_mk_mag, df_mk_mag, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_dur = fn.merge_mk_results(df_complete_mk_dur, df_mk_dur, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_intra = fn.merge_mk_results(df_complete_mk_intra, df_mk_intra, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_event_mag = fn.merge_mk_results(df_complete_mk_event_mag, df_mk_event_mag, df_copy.iloc[0]['site_no'], data_range, quantile) 
            df_complete_mk_event_dur = fn.merge_mk_results(df_complete_mk_event_dur, df_mk_event_dur, df_copy.iloc[0]['site_no'], data_range, quantile)
            df_complete_mk_timing = fn.merge_mk_results(df_complete_mk_timing, df_mk_timing, df_copy.iloc[0]['site_no'], data_range, quantile)        
            
            # Merging metric results
            df_single_iteration = pd.DataFrame(data, index=['0'])
            df_single_iteration = pd.concat([df_single_iteration.reset_index(drop=True), hmf_per_month.reset_index(drop=True)], axis=1)
            df_complete_site_data = pd.concat([df_complete_site_data.reset_index(drop=True), df_single_iteration.reset_index(drop=True)], axis=0)
        
    return df_complete_site_data, df_complete_mk_mag, df_complete_mk_dur, df_complete_mk_intra, df_complete_mk_event_mag, df_complete_mk_event_dur, df_complete_mk_timing

# For testing purposes, to run this cell independently
single_site_result = single_site_data(df, fn.QUANTILE_LIST, fn.DATA_RANGE_LIST)

'''
try:
    with pd.ExcelWriter('df_single_site.xlsx') as writer:
        df_complete_site_data.to_excel(writer, sheet_name='site_metrics', index=False)
        df_complete_mk_mag.to_excel(writer, sheet_name='mk_magnitude', index=False)
        df_complete_mk_intra.to_excel(writer, sheet_name='mk_intra', index=False)
        df_complete_mk_dur.to_excel(writer, sheet_name='mk_duration', index=False)
except Exception as e:
    print(e)
'''
   
fn.single_site_report(single_site_result[0])
df_complete_site_data = fn.gages_2_filtering(single_site_result[0])
#fn.save_data(df_complete_site_data, df_complete_mk_mag, df_complete_mk_dur, df_complete_mk_intra, 'TEST')

Site No: 0    11447650
1    11447650
2    11447650
0    11447650
Name: site_no, dtype: object
Analyzed Range: 30
30
50
50
Quantile: 0.90
0.95
0.90
0.95
Valid: True
True
True
True
% Missing: 0
0
0
0
90%: 48800.0
69000.0
50600.0
69500.0
HMF Years: 20
15
33
27
Annual Duration: 15.0
 1.5
14.0
 2.5
Event Duration: 8.50
1.50
6.25
2.50
Event HMF: 0.191078
0.012478
0.162024
0.015658
Inter-annual Frequency: 66.667
50.000
66.000
54.000%
Intra-annual Frequency: 2.0
0.5
2.0
1.0
Total HMF in km^3/year: 0.049421
0.016147
0.046485
0.016147
Center of Mass: 146.5
155.0
144.0
145.0
6 Month HMF in km^3/year: 0.050644
0.016392
0.047464
0.017126
3 Month HMF in km^3/year: 0.053091
0.022264
0.048932
0.021530


## CORE: Multi-Site Filtering<br>
This function creates the list of sites to analyze by filtering the complete list of state sites with 00060_Mean data down to those that lie within a specific watershed boundary using decimal long/lat positional data and a region shapefile (e.g. state or watershed boundary)<br><br>

Controls:

In [12]:
# This is only needed when running the following cell independently
shapefile_path = 'ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp'

Site List Generation:

In [26]:
def filter_state_site(shapefile_path: str, state_uri: str):
    """Creates a list of sites with over 50 years of 00060_Mean streamflow data for a given region"""
    # Request page from USGS site, ignore all informational lines
    response = requests.get(state_uri)
    data = response.text
    lines = data.splitlines()
    lines = [line for line in lines if not line.startswith('#')]

    # Create dataframe where site_no is a list of all sites in a state with 00060 data
    tsd = "\n".join(lines)
    df = pd.read_csv(StringIO(tsd), sep='\t')
    df_state_sites = df.iloc[1:]
        
    # Filter out sites outside of HU boundary
    if fn.SORT_BY_WB:
        shapefile = gpd.read_file(shapefile_path)
        shapefile = shapefile.to_crs(epsg=4326)
        df_state_sites['geometry'] = [Point(lon, lat) for lon, lat in zip(df_state_sites['dec_long_va'], df_state_sites['dec_lat_va'])]
        gdf_data = gpd.GeoDataFrame(df_state_sites, crs=shapefile.crs)
        df_state_sites = gpd.sjoin(gdf_data, shapefile, predicate='within')
            
    #print(df_state_sites.columns.to_list())
    #print(df_state_sites)
    
    return df_state_sites

df_state_sites = filter_state_site(shapefile_path, fn.SITES_URI)
print(f'Total Sites: {len(df_state_sites)} in the state of {fn.STATE_CODE.upper()} in the given WB')
site_list = df_state_sites['site_no'].to_list()
print(site_list)

Total Sites: 72 in the state of ME in the given WB
['01010000', '01010070', '01010500', '01011000', '01011500', '01013500', '01014000', '01015800', '01017000', '01017060', '01017290', '01017550', '01017960', '01018009', '01018035', '01018500', '01019000', '01021000', '01021470', '01021480', '01022500', '01022840', '01027200', '01027240', '01029100', '01029200', '01029500', '01030350', '01030500', '01031300', '01031450', '01031500', '01031510', '01033000', '01034000', '01034500', '01037000', '01037380', '01038000', '01042500', '01043500', '01044550', '01046000', '01046500', '01047000', '01047150', '01047200', '01048000', '01048220', '01049265', '01049500', '01052500', '01053500', '01053600', '01054000', '01054114', '01054200', '01054300', '01054500', '01055000', '01055500', '01057000', '01059000', '01060000', '01063310', '01064140', '01064500', '01066000', '01067950', '01068910', '01069500', '01069700']


## CORE: National Metrics Dataset
Generates a nationwide dataset for all sites meeting datarange criteria (lengthy runtime!)

In [6]:
huc2_gdf = gpd.read_file('ShapeFiles/HUC2/_Master_HUC2/master_huc2.shp')
huc4_gdf = gpd.read_file('ShapeFiles/HUC4/_Master_HUC4/master_huc4.shp')
aq_gdf = gpd.read_file('ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp')

In [7]:
state_limit = 50
dataset_name = 'National_Metrics'
shapefile_path = 'ShapeFiles/Lower48/lower48.shp'

# States with few sites for testing purposes
test_state_list = ['ME', 'DE']

# Set to False to ignore the blacklist and analyze all sites. This could be occasionally
# worth doing as USGS could update site data making it valid in the future
allow_blacklist = False

curr_blacklist = []
try:
    with open('Prelim_Data/_National_Metrics/National_Sites_Blacklist.txt', 'r') as file:
        curr_blacklist = file.read().split(', ')
        curr_blacklist = [x.replace("'", "") for x in curr_blacklist]
except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'Prelim_Data/_National_Metrics/National_Sites_Blacklist.txt'


In [ ]:
df_natl_metrics = pd.DataFrame()
df_natl_mk_mag = pd.DataFrame()
df_natl_mk_dur = pd.DataFrame()
df_natl_mk_intra = pd.DataFrame()
df_natl_mk_event_mag = pd.DataFrame()
df_natl_mk_event_dur = pd.DataFrame()
df_natl_mk_timing = pd.DataFrame()

#natl_blacklist = []

# Use fn.STATE_LIST for full dataset generation
for state_index, state in enumerate(fn.STATE_LIST):
    if state_index >= state_limit: break
    
    print(f'[---Working on {state}---]')
    state_uri = fn.create_state_uri(state, fn.PARAM_CODE)
    df_state_sites = filter_state_site(shapefile_path, state_uri)
    df_state_sites = df_state_sites.reset_index()
    print(f'Total Sites: {len(df_state_sites)} in the state of {state}')
    
    # Modified version of the create_multi_site_data() function
    for site_index, row in df_state_sites.iterrows():
        
        #if allow_blacklist and str(row['site_no']) in curr_blacklist:
            #print(f'IGNORED: Site {row["site_no"]} is in aquifer blacklist')
            #continue
        
        df = nwis.get_record(sites=row['site_no'], service=fn.SERVICE, parameterCD=[fn.PARAM_CODE, fn.TIDAL_CODE], start=fn.DEFAULT_START, end=fn.DEFAULT_END)
        df = df.reset_index()
        print(f'Working on {state} site {site_index + 1}/{len(df_state_sites)} ({row["site_no"]})')
        
        # No data at all
        if df.empty:
            #natl_blacklist.append(row['site_no'])
            print(f'IGNORED: No data for site {row["site_no"]}')
            continue
        
        # Important for one outlet gauge in TGC aquifer (and maybe others)
        if '00060_radar sensor_Mean' in df.columns and '00060_Mean' not in df.columns:
            df.rename(columns={'00060_radar sensor_Mean': '00060_Mean'}, inplace=True)
        
        # Merge tidal data before beginning analysis
        df = fn.merge_tidal(df)
        
        start = df['datetime'].min().date()
        end = df['datetime'].max().date()
        range = round((end - start).days / 365.25, 1)
        
        # Removed to create datasets with all 00060_Mean data regardless of available range
        '''if range < fn.MIN_DATA_PERIOD and allow_blacklist:
            natl_blacklist.append(row['site_no'])
            print(f'IGNORED: Not enough data for site {row["site_no"]}')
            continue'''
        
        point = Point(row['dec_long_va'], row['dec_lat_va'])
        #state_code = row['station_nm'].strip()[-2:]
        
        huc2 = huc4 = aquifer = 'NA'
        # HUC2 assignment
        for i, geo_row in huc2_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                huc2 = geo_row['huc2_code']
                continue
            
        # HUC4 assignment
        for i, geo_row in huc4_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                huc4 = geo_row['huc4_code']
                continue
            
        # Aquifer assignment    
        for i, geo_row in aq_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                aquifer = geo_row['aq_name']
                continue        
        
        # A few very broken sites with almost no data can have 0 hmf years and cause errors (i.e. '03592000')
        # so we'll catch these and add them to the site blacklist
        try:
            df_single_site_metric, df_mk_mag, df_mk_dur, df_mk_intra, df_mk_event_mag, df_mk_event_dur, df_mk_timing = single_site_data(df, fn.QUANTILE_LIST, fn.DATA_RANGE_LIST)
            add_data = {'dec_lat_va': row['dec_lat_va'], 'dec_long_va': row['dec_long_va'], 'data_start': start, 'data_end': end, 'total_record': range, 
                        'state': state, 'huc2_code': huc2, 'huc4_code': huc4, 'within_aq': aquifer}			
            add_data = pd.DataFrame(add_data, index=['0'])
        except Exception as e:
            #natl_blacklist.append(row['site_no'])
            print(f"ERROR: Single site data failure for site {row['site_no']}:\n{e}")
            continue
        
        add_data = pd.DataFrame(np.tile(add_data.values, (len(df_single_site_metric), 1)), columns=add_data.columns)
        df_single_site_metric = pd.concat([df_single_site_metric.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_mag = pd.concat([df_mk_mag.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_dur = pd.concat([df_mk_dur.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_intra = pd.concat([df_mk_intra.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_event_mag = pd.concat([df_mk_event_mag.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_event_dur = pd.concat([df_mk_event_dur.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        df_mk_timing = pd.concat([df_mk_timing.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
        
        # Append single site data to multi-site dataframes
        df_natl_metrics = pd.concat([df_natl_metrics, df_single_site_metric], ignore_index=True)
        df_natl_mk_mag = pd.concat([df_natl_mk_mag, df_mk_mag], ignore_index=True)
        df_natl_mk_dur = pd.concat([df_natl_mk_dur, df_mk_dur], ignore_index=True)
        df_natl_mk_intra = pd.concat([df_natl_mk_intra, df_mk_intra], ignore_index=True)
        df_natl_mk_event_mag = pd.concat([df_natl_mk_event_mag, df_mk_event_mag], ignore_index=True)
        df_natl_mk_event_dur = pd.concat([df_natl_mk_event_dur, df_mk_event_dur], ignore_index=True)
        df_natl_mk_timing = pd.concat([df_natl_mk_timing, df_mk_timing], ignore_index=True)
        
df_natl_metrics = fn.gages_2_filtering(df_natl_metrics)

# Create national site blacklist
'''try:
    with open(f'Prelim_Data/National_Sites_Blacklist.txt', 'w') as f:
        natl_blacklist = ["'" + str(x) + "'" for x in natl_blacklist]        
        f.write(', '.join(natl_blacklist))
except Exception as e:
    print(e)'''

try:    
    fn.save_data(df_natl_metrics, df_natl_mk_mag, df_natl_mk_dur, df_natl_mk_intra, df_natl_mk_event_mag, df_natl_mk_event_dur, df_natl_mk_timing, dataset_name)
except Exception as e:
    print(f'Error saving data: {e}')

[---Working on ME---]
Total Sites: 72 in the state of ME
Working on ME site 1/72 (01010000)
Working on ME site 2/72 (01010070)
Working on ME site 3/72 (01010500)
Working on ME site 4/72 (01011000)
Working on ME site 5/72 (01011500)
ERROR: Single site data failure for site 01011500:
'00060_Mean'
Working on ME site 6/72 (01013500)
Working on ME site 7/72 (01014000)
Working on ME site 8/72 (01015800)
Working on ME site 9/72 (01017000)
Working on ME site 10/72 (01017060)
Working on ME site 11/72 (01017290)
Working on ME site 12/72 (01017550)
Working on ME site 13/72 (01017960)
Working on ME site 14/72 (01018009)
Working on ME site 15/72 (01018035)
Working on ME site 16/72 (01018500)
Working on ME site 17/72 (01019000)
Working on ME site 18/72 (01021000)
Working on ME site 19/72 (01021470)
Working on ME site 20/72 (01021480)
Working on ME site 21/72 (01022500)
Working on ME site 22/72 (01022840)
Working on ME site 23/72 (01027200)
Working on ME site 24/72 (01027240)
Working on ME site 25/72

## EXTRA: Outlet Gauges Dataset
Uses <code>outlet_gauges.xlsx</code> to generate a custom dataset with only outlet gauges present 

In [47]:
df_outlets = pd.read_excel('Prelim_Data/_Outlet_Gauges/outlet_gauges.xlsx', dtype={'site_no': str})
test_limit = 999
#print(df_outlets.duplicated(subset=['site_no']))

In [46]:
df_outlet_metrics = pd.DataFrame()
df_outlet_mk_mag = pd.DataFrame()
df_outlet_mk_dur = pd.DataFrame()
df_outlet_mk_intra = pd.DataFrame()

for i, row in df_outlets.iterrows():
    if i >= test_limit: break
    print(f'[---Working on site {row["site_no"]}---]')
    df = nwis.get_record(sites=row['site_no'], service=fn.SERVICE, parameterCD=fn.PARAM_CODE, start=fn.DEFAULT_START, end=fn.DEFAULT_END)
    df = df.reset_index()
    
    if '00060_radar sensor_Mean' in df.columns and '00060_Mean' not in df.columns:
        df.rename(columns={'00060_radar sensor_Mean': '00060_Mean'}, inplace=True)
    
    start = df['datetime'].min().date()
    end = df['datetime'].max().date()
    range = round((end - start).days / 365.25, 1)
    
    aquifer = row['aquifer']
    
    try:
        df_single_site_metric, df_mk_mag, df_mk_dur, df_mk_intra = single_site_data(df, fn.QUANTILE_LIST, fn.DATA_RANGE_LIST)
        add_data = {'data_start': start, 'data_end': end, 'total_record': range, 'aquifer': aquifer}			
        add_data = pd.DataFrame(add_data, index=['0'])
    except Exception as e:
        print(f"ERROR: Single site data failure for site {row['site_no']}:\n{e}")
        continue
    
    add_data = pd.DataFrame(np.tile(add_data.values, (len(df_single_site_metric), 1)), columns=add_data.columns)
    df_single_site_metric = pd.concat([df_single_site_metric.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
    df_mk_mag = pd.concat([df_mk_mag.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
    df_mk_dur = pd.concat([df_mk_dur.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
    df_mk_intra = pd.concat([df_mk_intra.reset_index(drop=True), add_data.reset_index(drop=True)], axis=1)
    
    df_outlet_metrics = pd.concat([df_outlet_metrics, df_single_site_metric], ignore_index=True)
    df_outlet_mk_mag = pd.concat([df_outlet_mk_mag, df_mk_mag], ignore_index=True)
    df_outlet_mk_dur = pd.concat([df_outlet_mk_dur, df_mk_dur], ignore_index=True)
    df_outlet_mk_intra = pd.concat([df_outlet_mk_intra, df_mk_intra], ignore_index=True)
    
df_outlet_metrics = fn.gages_2_filtering(df_outlet_metrics)

try:
    fn.save_data(df_outlet_metrics, df_outlet_mk_mag, df_outlet_mk_dur, df_outlet_mk_intra, 'Outlet_Metrics')
except Exception as e:
    print(e)



[---Working on site 09519800---]
[---Working on site 09521100---]
[---Working on site 11303500---]
[---Working on site 11447650---]
[---Working on site 14105700---]
[---Working on site 07316000---]
[---Working on site 02330000---]
[---Working on site 02320500---]
[---Working on site 02236000---]
[---Working on site 02296750---]
[---Working on site 02292900---]
[---Working on site 02198500---]
[---Working on site 02169500---]
[---Working on site 02129000---]
[---Working on site 13269000---]
[---Working on site 08114000---]
[---Working on site 08033500---]
[---Working on site 08066500---]
[---Working on site 08068000---]
[---Working on site 08116650---]
[---Working on site 08162000---]
[---Working on site 08176500---]
[---Working on site 08211000---]
[---Working on site 08164000---]
[---Working on site 08188500---]
[---Working on site 08030500---]
[---Working on site 08013500---]
[---Working on site 07378500---]
[---Working on site 07375000---]
[---Working on site 02489500---]
[---Workin

## EXTRA: Nationwide Validity Dataset
Generates a dataset for every water gauge with 00060_Mean data in the contiguous US indicating gauge validity for this study

In [10]:
shapefile_path = 'ShapeFiles/Lower48/lower48.shp'
test_state_list = ['ME', 'DE']
test_limit = 50

In [11]:

df_natl_validity = pd.DataFrame()

for i, state in enumerate(fn.STATE_LIST):
    if i >= test_limit: break
    print(f'[---Working on {state}---]')
    state_uri = fn.create_state_uri(state, fn.PARAM_CODE)
    df_state_sites = filter_state_site(shapefile_path, state_uri)
    print(f'Total Sites: {len(df_state_sites)} in the state of {state}')
    
    ignored_count = 0
    for loc, row in df_state_sites.iterrows():
        df = nwis.get_record(sites=row['site_no'], service=fn.SERVICE, parameterCD=[fn.PARAM_CODE, fn.TIDAL_CODE], start=fn.DEFAULT_START, end=fn.DEFAULT_END)
        df = df.reset_index()
        
        if df.empty:
            ignored_count += 1
            continue
        
        if '00060_radar sensor_Mean' in df.columns and '00060_Mean' not in df.columns:
            df.rename(columns={'00060_radar sensor_Mean': '00060_Mean'}, inplace=True)
        
        df = fn.merge_tidal(df)
        
        start = df['datetime'].min().date()
        end = df['datetime'].max().date()
        total_record = round((end - start).days / 365.25, 1)
        
        valid_range_30 = total_record >= 27
        valid_range_50 = total_record >= 45
                
        date_threshold_30 = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * 30)
        df_30 = df[df['datetime'].dt.date >= date_threshold_30]
        missing_30 = fn.validate(df_30, date_threshold_30, fn.DEFAULT_END)
        valid_30 = missing_30 < fn.MAX_MISSING_THRESHOLD
        
        date_threshold_50 = pd.to_datetime(fn.DEFAULT_END).date() - timedelta(days=365.25 * 50)
        df_50 = df[df['datetime'].dt.date >= date_threshold_50]
        missing_50 = fn.validate(df_50, date_threshold_50, fn.DEFAULT_END)
        valid_50 = missing_50 < fn.MAX_MISSING_THRESHOLD
        
        missing_30 = round(missing_30, 5) * 100
        missing_50 = round(missing_50, 5) * 100
        
        data = {'site_no': str(row['site_no']), 'state': state, 'date_range_30': valid_range_30, 'data_cont_30': valid_30, 'missing_30': missing_30,
                'date_range_50': valid_range_50, 'data_cont_50': valid_50, 'missing_50': missing_50, 'start': start, 'end': end, 
                'total_record': total_record, 'dec_lat_va': row['dec_lat_va'], 'dec_long_va': row['dec_long_va']}
        
        df_natl_validity = pd.concat([df_natl_validity, pd.DataFrame(data, index=['0'])], axis=0, ignore_index=True)
        
    print(f'Ignored {ignored_count} sites ({round((ignored_count / len(df_state_sites)) * 100, 2)}%)')        
        
try:
    with pd.ExcelWriter('Prelim_Data/National_Validity.xlsx') as writer:
        df_natl_validity = df_natl_validity.drop_duplicates(subset=['site_no'], keep='first')
        df_natl_validity.to_excel(writer, sheet_name='national_validity', index=False) 
except Exception as e:
    print(e)

[---Working on AL---]
Total Sites: 177 in the state of AL
Ignored 30 sites (16.95%)
[---Working on AZ---]
Total Sites: 218 in the state of AZ
Ignored 9 sites (4.13%)
[---Working on AR---]
Total Sites: 139 in the state of AR
Ignored 12 sites (8.63%)
[---Working on CA---]
Total Sites: 484 in the state of CA
Ignored 25 sites (5.17%)
[---Working on CO---]
Total Sites: 348 in the state of CO
Ignored 14 sites (4.02%)
[---Working on CT---]
Total Sites: 75 in the state of CT
Ignored 4 sites (5.33%)
[---Working on DE---]
Total Sites: 36 in the state of DE
Ignored 2 sites (5.56%)
[---Working on FL---]
Total Sites: 436 in the state of FL
Ignored 9 sites (2.06%)
[---Working on GA---]
Total Sites: 308 in the state of GA
Ignored 5 sites (1.62%)
[---Working on ID---]
Total Sites: 231 in the state of ID
Ignored 9 sites (3.9%)
[---Working on IL---]
Total Sites: 193 in the state of IL
Ignored 4 sites (2.07%)
[---Working on IN---]
Total Sites: 207 in the state of IN
Ignored 16 sites (7.73%)
[---Working o

## EXTRA: HUC2/HUC4/Aquifer Master Shapefile Creation
The following code combines HUC2, HUC4, and Aquifer shapefiles into master shapefiles, respectively. It's not necessary to run if these files already exist within the ShapeFiles directory

In [14]:
huc2_path = 'ShapeFiles/HUC2'
huc4_path = 'ShapeFiles/HUC4'
aquifer_path = 'ShapeFiles/Aquifers'
aq_ext = '.shp'
huc2 = 'WBDHU2.shp'
huc4 = 'WBDHU4.shp'

create_huc2 = False
create_huc4 = False
create_aquifer = True

huc2_gdf = gpd.GeoDataFrame()
huc4_gdf = gpd.GeoDataFrame()
aquifer_df = gpd.GeoDataFrame()

# HUC2's
if create_huc2:
    for root, dirs, files in os.walk(huc2_path):
        if os.path.basename(root).startswith('WBD_'):
            code = root[-2:]
            if huc2 in files:
                shape = gpd.read_file(os.path.join(root, huc2))
                shape['huc2_code'] = code
                huc2_gdf = pd.concat([huc2_gdf, shape], ignore_index=True)

    huc2_gdf = huc2_gdf.to_crs(4269)
    huc2_gdf.to_file('ShapeFiles/HUC2/_Master_HUC2/master_huc2.shp')

# HUC4's    
if create_huc4:
    for root, dirs, files in os.walk(huc4_path):
        if os.path.basename(root).startswith('NHD_H_'):
            code = root[-4:]
            if huc4 in files:
                shape = gpd.read_file(os.path.join(root, huc4))
                shape['huc4_code'] = code
                huc4_gdf = pd.concat([huc4_gdf, shape], ignore_index=True)

    huc4_gdf = huc4_gdf.to_crs(4269)
    huc4_gdf.to_file('ShapeFiles/HUC4/_Master_HUC4/master_huc4.shp')
    
# Aquifers
if create_aquifer:
    for root, dirs, files in os.walk(aquifer_path):
        for file in files:
            if file.endswith(aq_ext) and dirs:
                if file.startswith('Penn'):
                    shape = gpd.read_file(os.path.join(root, file))
                    shape = shape.iloc[[17]]                    
                else:
                    shape = gpd.read_file(os.path.join(root, file))
                    
                shape = shape.to_crs(4269)
                shape['aq_name'] = file[:-4]
                aquifer_df = pd.concat([aquifer_df, shape], ignore_index=True)
                
    aquifer_df = aquifer_df.to_crs(4269)
    aquifer_df.to_file('ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp')

## EXTRA: HUC2/4/Aquifer Sorting
Takes a national validity or national metrics dataset (any dataset with dec_lat_va and dec_long_va data), and adds corresponding HUC2, HUC4, and Aquifer columns indicating a water guages presence or lack thereof in each of these boundaries.

<strong>NOTE:</strong> This feature has already been implemented directly into the National Metrics dataset creation. Its only real use is on National Validity datasets

In [14]:
path = 'Prelim_Data/_National_Validity/'
datasets = ['National_Validity.xlsx']
huc2_gdf = gpd.read_file('ShapeFiles/HUC2/_Master_HUC2/master_huc2.shp')
huc4_gdf = gpd.read_file('ShapeFiles/HUC4/_Master_HUC4/master_huc4.shp')
aq_gdf = gpd.read_file('ShapeFiles/Aquifers/_Master_Aquifer/master_aquifer.shp')

In [15]:
for dataset in datasets:
    temp = pd.DataFrame()
    
    df = pd.read_excel(f'{path}{dataset}', dtype={'site_no': str})
    sheets = pd.ExcelFile(f'{path}{dataset}')
    sheet = sheets.sheet_names[0]          

    for i, row in df.iterrows():
        add_data = pd.DataFrame({'site_no': str(row['site_no']), 'huc2_code': 'NA', 'huc4_code': 'NA', 'aquifer': 'NA'}, index=['0'])
        point = Point(row['dec_long_va'], row['dec_lat_va'])
        
        # HUC2 assignment
        for j, geo_row in huc2_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                add_data["huc2_code"] = geo_row['huc2_code']
                continue
            
        # HUC4 assignment
        for j, geo_row in huc4_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                add_data["huc4_code"] = geo_row['huc4_code']
                continue
            
        # Aquifer Assignment
        for j, geo_row in aq_gdf.iterrows():
            if geo_row['geometry'].contains(point):
                add_data["aquifer"] = geo_row['aq_name']
                continue
            
        temp = pd.concat([temp, add_data], axis=0, ignore_index=True)
    
    df = pd.merge(df, temp, on='site_no', validate='1:1')
    
    with pd.ExcelWriter(f'{path}{dataset}', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet, index=False)      

## EXTRA: National Dataset Splitting by Aquifer
This script splits a national metric dataset or datasets into smaller per-aquifer datasets

In [16]:
natl_path = 'Prelim_Data/_National_Metrics'

# National datasets to split
datasets = ['National_Metrics_30_90.xlsx', 'National_Metrics_30_95.xlsx', 'National_Metrics_50_90.xlsx', 'National_Metrics_50_95.xlsx']
# The datasets to generate from the national dataset
target_aquifers = cl.ALL_AQUIFERS
sheet_names = ['site_metrics', 'mk_magnitude', 'mk_duration', 'mk_intra_annual']

In [17]:
# Iterate over national datasets
for dataset in datasets:
    df_list = []
    for sheet in sheet_names:
        df = pd.read_excel(f'{natl_path}/{dataset}', sheet_name=sheet, dtype=fn.DATASET_DTYPES)
        df_list.append(df)

    # Iterate over target aquifers
    for aquifer in target_aquifers:
        save_path = f"{aquifer.datasets_dir}/{aquifer.name}_{dataset[-10:-8]}_{dataset[-7:-5]}.xlsx"
        for i, df in enumerate(df_list):
            df = df[df['huc4_code'].isin(aquifer.huc4s)]
            
            # Append new sheets to existing file
            if os.path.exists(save_path):
                with pd.ExcelWriter(save_path, mode='a', if_sheet_exists='replace') as writer:
                    df.to_excel(writer, sheet_name=sheet_names[i], index=False)
            # Create file if it doesn't exist (first iteration)                        
            else: 
                with pd.ExcelWriter(save_path, mode='w') as writer:
                    df.to_excel(writer, sheet_name=sheet_names[i], index=False) 